In [1]:
import awkward as ak
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist # coffea.hist is deprecated as of Jan 2023!

# this avoids some warnings due to using older NanoAOD ntuples
NanoAODSchema.warn_missing_crossrefs = False

The code below introduces some basic concepts for writing code using Coffea.  

There are three primary pieces to the Coffea code:  
1. The *processor*, which contains all of the analysis cuts and fills the histogram in the _process_ function. 
2. The second cell defines the files we want to run over and then runs the code using run_uproot_job. 
3. After we run the processor, we can then plot any of the histograms we have generated.

To test any changes you make to the histograms, you will have to rerun each of the three cells below.

In [2]:
fname = "root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTGamma_1l.root"
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

In [3]:
events.fields

['GenJet',
 'luminosityBlock',
 'LHEWeight',
 'event',
 'Muon',
 'GenPart',
 'Generator',
 'Jet',
 'Photon',
 'fixedGridRhoFastjetAll',
 'Flag',
 'Pileup',
 'Electron',
 'run',
 'PSWeight',
 'GenMET',
 'LHEScaleWeight',
 'HLT',
 'MET',
 'LHEPdfWeight']

In [4]:
events.Electron.fields

['dxy', 'dz', 'eta', 'mass', 'phi', 'pt', 'charge', 'cutBased']

In [5]:
events.Electron.eta.fields

[]

In [6]:
class TTGammaCutflow(processor.ProcessorABC):
    def process(self, events):
        ### The process function is where most of the work happens. As we'll see below, this is
        ### where the main analysis work happens (object cuts, event selections, filling histograms)

        ## To access variables from the ntuples, use the "events" object
        ## The dataset name is part of events.metadata
        dataset = events.metadata['dataset']
        
        ## The coffea NanoEventSchema packages all muon variables (columns) into the events.Muon object
        ## Each variable can be accessed using muons.key_name
        muons = events.Muon
        electrons = events.Electron
        
        ######
        
        # Select muons with pt >30, eta < 2.4, tight ID, and relIso < 0.15
        muonSelectTight = ((muons.pt > 30) & (muons.eta < 2.4) & (muons.tightId) & (muons.pfRelIso04_all < 0.15))

        # Apply the selection to muons using the array[mask] syntax. 
        # tightMuons only includes the muons that pass the tight selection we defined
        tightMuons = muons[muonSelectTight]
        
        ######
        ### Step 2. Define your own selection for electrons. 
        ### Note that the ID variable names will be different. 
        ### Either remove the ID and iso variables or replace them with the correct electron values.
        ######
        
        eleSelectTight = ((electrons.pt >30) & (abs(electrons.eta)<2.4) & (electrons.cutBased))
        tightElectrons = electrons[eleSelectTight]

        ######
        
        # Select events with exactly one tight muon. 
        eventSelectionMuon = (ak.num(tightMuons)==1)
        
        ######
        ### Step 3. Define a second event selection requiring events with no muons and exactly one electron
        ###### 
        eventSelectionEle = ((ak.num(tightElectrons)==1) & (ak.num(tightMuons)==0))

        ######
        # we will make a few different histograms, and store them in this dictionary
        output = {}

        # Define and fill a muon_pt histogram using the tightMuons in events that pass our selection 
        # Note that ak.flatten() is required when filling a histogram to remove the jaggedness
        output['muon_pt'] = hist.Hist(
            hist.axis.Regular(40, 0, 200, name="pt", label="Muon $p_{T}$ [GeV]")
        )
        output['muon_pt'].fill(pt=ak.flatten(tightMuons[eventSelectionMuon].pt))
        
        ######
        ### Step 4. Fill an ele_pt histogram
        ###### 
        output['ele_pt'] = hist.Hist(
            hist.axis.Regular(40, 0, 200, name="pt", label="Muon $p_{T}$ [GeV]")
        )
        output['ele_pt'].fill(pt=ak.flatten(tightElectrons[eventSelectionEle].pt))
        ######
        
        
        # we can return any nested dictionary of histograms or numbers
        # by having one set of histograms per dataset we can properly scale
        # the simulation by its cross sections later
        return {dataset: output}

    def postprocess(self, accumulator):
        return accumulator


In [7]:
# Define files to run over
fileset = {
    "TTGamma": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTGamma_1l.root"
    ],
    "TTbar": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTbar_1l.root"
    ],
}

# Run Coffea code using uproot
output = processor.run_uproot_job(
    fileset,
    "Events",
    TTGammaCutflow(),
    processor.iterative_executor,
    {"schema": NanoAODSchema},
)

Output()

Output()

In [8]:
from ttgamma.utils.crossSections import lumis, crossSections
import uproot

myCrossSections = {}
myCrossSections['TTGamma'] = crossSections['TTGamma_SingleLept']
myCrossSections['TTbar'] = crossSections['TTbarPowheg_Semilept']

for dataset_name, histos in output.items():
    output[dataset_name]["InputEventCount"] = processor.value_accumulator(int)
    for filename in fileset[dataset_name]:
        with uproot.open(filename) as fhandle:
            output[dataset_name]["InputEventCount"] += (
            fhandle["hEvents"].values()[2] - fhandle["hEvents"].values()[0]
        )
                    
    for name, obj in histos.items():
        if isinstance(obj, hist.Hist):
            obj *= myCrossSections[dataset_name] * lumis[2016] / output[dataset_name]["InputEventCount"].value

SyntaxError: invalid syntax (processor.py, line 97)

In [ ]:
output.items()

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 8))
# Plot the muon_pt histogram, overlaying the distribution for each dataset
# Sometimes have to run this cell twice for the plot to appear
output['TTGamma']['muon_pt'].plot1d(ax=ax1, label='ttgamma');
output['TTbar']['muon_pt'].plot1d(ax=ax1, label='ttbar');
ax1.legend();

######
### Step 5. Plot the ele_pt histogram
###### 
output['TTGamma']['ele_pt'].plot1d(ax=ax2, label='ttgamma');
output['TTbar']['ele_pt'].plot1d(ax=ax2, label='ttbar');
ax2.legend();
######

If you want to look at other files, feel free to edit _fileset_ above using the skimmed datasets here: /store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles

In [ ]:
output['TTGamma']['muon_pt'].plot1d(density=True, label='ttgamma');
output['TTbar']['muon_pt'].plot1d(density=True, label='ttbar');
plt.legend();

In [ ]:
# plot the electron pT
output['TTGamma']['ele_pt'].plot1d(density=True, label='ttgamma');
output['TTbar']['ele_pt'].plot1d(density=True, label='ttbar');
plt.legend();

Now that you've seen some of the basics of histogramming, object cuts, and event selection, let's go through how to apply weights to the histograms.

In [ ]:
import uproot
import numpy as np


from coffea.lookup_tools import dense_lookup
from coffea.analysis_tools import Weights

class TTGammaWeights(processor.ProcessorABC):
    def __init__(self):
        
        # There are several utilities within coffea that can help us apply weights and systematics to the code
        # We use uproot to open the root file containing the electron ID scale factors:
        ele_id_file = uproot.open('ttgamma/scalefactors/MuEGammaScaleFactors/ele2016/2016LegacyReReco_ElectronTight_Fall17V2.root')
        # The dense_lookup tools in Coffea make it easy to extract the histogram (named EGamma_SF2D in this case)
        # with the weights and their errors:
        self.ele_id_sf = dense_lookup.dense_lookup(
            ele_id_file["EGamma_SF2D"].values(),
            (
                ele_id_file["EGamma_SF2D"].axis(0).edges(),
                ele_id_file["EGamma_SF2D"].axis(1).edges()
            )
        )
        self.ele_id_err = dense_lookup.dense_lookup(
            ele_id_file["EGamma_SF2D"].variances()**0.5,
            (
                ele_id_file["EGamma_SF2D"].axis(0).edges(),
                ele_id_file["EGamma_SF2D"].axis(1).edges()
            )
        )
        
        # We can also pre-define our histograms as shown below, but each process() call
        # needs to create a "fresh" copy of the histogram or else you will count events multiple times
        # when the outputs are merged by the coffea tools. That's why we define the `make_output` factory function

        # The systematic axis will be used to keep track of whether we are using the nominal weights
        # or the weights shifted up/down by their errors
        systematic_axis = hist.axis.StrCategory([], name="systematic", label="Systematic uncertainty", growth=True)
        
        ele_pt_axis  = hist.axis.Regular(40, 0, 200, name="pt", label="Electron $p_{T}$ [GeV]")

        self.make_output = lambda: {
            'ele_pt' : hist.Hist(systematic_axis, ele_pt_axis),
        }
    
    def process(self, events):

        output = self.make_output()

        dataset = events.metadata['dataset']

        electrons = events.Electron
        
        #Define tight electron selection
        electronSelectTight = (
            (electrons.pt>35) & 
            (abs(electrons.eta)<2.1) & 
            ((abs(electrons.eta) < 1.4442) | (abs(electrons.eta) > 1.566)) &      
            (electrons.cutBased>=4)
        )
        # Apply selection
        tightElectron = electrons[electronSelectTight]
        #Define event selection
        eventSelection = (ak.num(tightElectron) == 1)


        # Here, we look up the scale factors and errors for each electron using the electron eta and pt
        eleID = self.ele_id_sf(tightElectron.eta, tightElectron.pt)
        eleIDerr = self.ele_id_err(tightElectron.eta, tightElectron.pt)
        
        #To get an event-level weight, multiply the SF for each electron in the event
            # SF : scale factor
        #The axis=-1 option means we are multiplying the innermost values (electrons per event in this case)
        eleSF = ak.prod(eleID, axis=-1) # product of elements along axis
        eleSFUp = ak.prod(eleID+eleIDerr, axis=-1)
        eleSFDown = ak.prod(eleID-eleIDerr, axis=-1)
        print(eleSF, eleSFUp, eleSFDown)
    
        # The Weights object is a container that handles the bookkeeping 
        # for event weights and associated systematic shifts.
        # The argument for the Weights object is the number of events we are processing
        weights = Weights(len(events))

        # Add the ele ID SF to the weights object
        weights.add('eleEffWeight',weight=eleSF,weightUp=eleSFUp,weightDown=eleSFDown)
        
        systList = ['noweight','nominal','eleEffWeightUp','eleEffWeightDown']

        for syst in systList:
           
            weightSyst = syst
            if syst=='nominal':
                weightSyst=None
                
            if syst=='noweight':
                evtWeight = np.ones(len(events))
            else:
                evtWeight = weights.weight(weightSyst) # overall event weight
                
            output['ele_pt'].fill(
                systematic=syst,
                pt=ak.flatten(tightElectron[eventSelection].pt),
                weight=evtWeight[eventSelection]
            )
           
        return {dataset: output}

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
# Define files to run over
fileset = {
    "TTGamma": [
        "root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTGamma_1l.root"
    ],
}


# Run Coffea code using uproot
output = processor.run_uproot_job(
    fileset,
    "Events",
    TTGammaWeights(),
    processor.iterative_executor,
    {"schema": NanoAODSchema},
)

In [ ]:
output.items()

In [ ]:
# Plot the pt distributions for each of the different systematics
fig, ax = plt.subplots(figsize=(10,10))
output['TTGamma']['ele_pt'].plot1d(overlay='systematic');
plt.legend();

In [ ]:
ak.prod([[1,2,3],[4,5]], axis=-1)

In [ ]:
ak.prod([])

In [ ]:
import ttgamma.scalefactors as sf

In [ ]:
sf.ele_id_sf # same as below

In [ ]:
TTGammaWeights().ele_id_sf